In [1]:
import pandas as pd

In [2]:
!pip install --upgrade accelerate

In [3]:
file_path_1 = './i2b2_parsing/parsed_gold_set_1.jsonl'
test_path = './i2b2_parsing/parsed_test.jsonl'
data = pd.read_json(file_path_1, lines=True)
test_data = pd.read_json(test_path, lines=True)

In [4]:
data

,text,spans,meta
0,\n\n\nRecord date: 2080-11-30\n\n\n\nReason fo...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '369-03', 'patient': '369', 'recor..."
1,\n\n\nRecord date: 2061-01-15\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '299-01', 'patient': '299', 'recor..."
2,\n\n\nRecord date: 2083-04-03\n\n\nBeaumont Ho...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '302-02', 'patient': '302', 'recor..."
3,\n\n\nRecord date: 2094-05-01\n\nCC: Annual ex...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '327-04', 'patient': '327', 'recor..."
4,"\n\n\nRecord date: 2094-06-12\n\nJune 12, 2094...","[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '297-04', 'patient': '297', 'recor..."
...,...,...,...
785,\n\n\nRecord date: 2135-09-08\n\n\n\tCARDIOLOG...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '108-04', 'patient': '108', 'recor..."
786,\n\n\nRecord date: 2067-11-24\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '103-01', 'patient': '103', 'recor..."
787,\n\n\nRecord date: 2068-01-25\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '103-02', 'patient': '103', 'recor..."
788,\n\n\nRecord date: 2068-09-21\n\nEDVISIT^84091...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex...","{'note_id': '103-03', 'patient': '103', 'recor..."


In [5]:
texts = data['text'].tolist() # type: ignore
test_texts = test_data['text'].tolist() # type: ignore
texts[0]

'\n\n\nRecord date: 2080-11-30\n\n\n\nReason for Visit\n\nOwen is a 63 y/o male here for evaluation of treatment. Doin relatively well. \n\n\n\nProblems\n\n\n\n      OA\n\n\n\n      LLE-PARTIALLY SEVERED-MULT. SURGERIES\n\n\n\n      IRRIDECTOMY\n\n\n\n      SKIN ULCER-DR Esposito\n\n\n\n      PAST SMOKER \n\n\n\n      HTN\n\n\n\n\n\nMedications\n\n\n\n      ASA       PO \n\n\n\n      Vitamin E        PO QD : 400 IU \n\n\n\n      ATENOLOL   25 MG PO QD\n\n\n\n      Lipitor (ATORVASTATIN)    10MG,  1 Tablet(s)  PO QD\n\n\n\n\n\nAllergies\n\n\n\n      NKDA    - NONE\n\n\n\n\n\nNarrative History\n\nTakes meds. No SEs. Denies vision change, headache, chest pain, SOB, light head, palptations. Denies loss of balance, strength or sensation. \n\n Pulm- no cough.  Occ wheeze. No SOB.\n\nGI- no nausea, vomitting, dyspepsia, reflux, abdo pain, diarrhea, constipation, melena, BRBPR.\n\nGU- asymp\n\nLocomotor- pain left knee/leg. \n\nSees Dr Esposito for chr ulcer. Has surgical boot on now. \n\nExer

In [6]:
spans = data['spans'].tolist() # type: ignore
test_spans = test_data['spans'].tolist() # type: ignore
spans[0]

[{'id': 'P0',
  'start': '16',
  'end': '26',
  'text': '2080-11-30',
  'TYPE': 'DATE',
  'comment': '',
  'label': 'DATE'},
 {'id': 'P1',
  'start': '48',
  'end': '52',
  'text': 'Owen',
  'TYPE': 'PATIENT',
  'comment': '',
  'label': 'PATIENT'},
 {'id': 'P2',
  'start': '58',
  'end': '60',
  'text': '63',
  'TYPE': 'AGE',
  'comment': '',
  'label': 'AGE'},
 {'id': 'P3',
  'start': '242',
  'end': '250',
  'text': 'Esposito',
  'TYPE': 'DOCTOR',
  'comment': '',
  'label': 'STAFF'},
 {'id': 'P4',
  'start': '854',
  'end': '862',
  'text': 'Esposito',
  'TYPE': 'DOCTOR',
  'comment': '',
  'label': 'STAFF'},
 {'id': 'P5',
  'start': '1664',
  'end': '1683',
  'text': 'William Seth Potter',
  'TYPE': 'DOCTOR',
  'comment': '',
  'label': 'STAFF'}]

In [7]:
model_nm = 'microsoft/deberta-v3-small'

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, convert_slow_tokenizer
tokz = AutoTokenizer.from_pretrained(model_nm, use_fast=True)

/Users/manibatra/mambaforge/envs/phi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/manibatra/mambaforge/envs/phi/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddi

In [9]:
def tok_function(x): return tokz(x['text'])

In [10]:
from datasets import Dataset
ds = Dataset.from_pandas(data)
test_ds = Dataset.from_pandas(test_data)

In [11]:
tok_ds = ds.map(tok_function, batched=True)
test_tok_ds = test_ds.map(tok_function, batched=True)
tok_ds

Dataset({
    features: ['text', 'spans', 'meta', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 790
})

In [12]:
tok_ds['text'][0]
tok_ds['input_ids'][0]


[1,
 9737,
 1043,
 294,
 89553,
 271,
 1554,
 271,
 1076,
 18037,
 270,
 3847,
 12980,
 269,
 266,
 7592,
 2982,
 320,
 795,
 2844,
 422,
 270,
 4291,
 265,
 1035,
 260,
 771,
 547,
 2936,
 371,
 260,
 16067,
 27122,
 1093,
 14360,
 271,
 71889,
 476,
 43993,
 662,
 45227,
 4272,
 271,
 1135,
 55111,
 260,
 662,
 96700,
 41800,
 273,
 81084,
 50938,
 67607,
 3021,
 67958,
 23042,
 68148,
 271,
 15746,
 74543,
 68452,
 109013,
 1349,
 1233,
 22438,
 63834,
 30561,
 15511,
 10397,
 829,
 15511,
 78381,
 877,
 3755,
 30840,
 336,
 34823,
 57903,
 1502,
 977,
 15502,
 15511,
 78381,
 115162,
 287,
 40883,
 14634,
 55275,
 10383,
 285,
 466,
 24393,
 261,
 376,
 20587,
 555,
 268,
 285,
 15511,
 78381,
 79673,
 27507,
 13975,
 341,
 69897,
 31775,
 3401,
 33315,
 20542,
 260,
 585,
 8628,
 268,
 260,
 11289,
 3933,
 2345,
 575,
 261,
 11141,
 261,
 4458,
 1427,
 261,
 97273,
 261,
 731,
 761,
 261,
 77214,
 27592,
 268,
 260,
 11289,
 3933,
 1265,
 265,
 1990,
 261,
 2067,
 289,
 10261,
 26

In [13]:
data['spans']

0      [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
1      [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
2      [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
3      [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
4      [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
                             ...                        
785    [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
786    [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
787    [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
788    [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
789    [{'id': 'P0', 'start': '16', 'end': '26', 'tex...
Name: spans, Length: 790, dtype: object

In [14]:
input_ids = tok_ds['input_ids'][0]
tokens = tokz.convert_ids_to_tokens(input_ids)
token_positions = tokz(tok_ds['text'][0], return_offsets_mapping=True)['offset_mapping']

for token, input_id, token_pos in zip(tokens, input_ids, token_positions):
    print(f'{token}: {input_id}: {token_pos}')

[CLS]: 1: (0, 0)
▁Record: 9737: (3, 9)
▁date: 1043: (9, 14)
:: 294: (14, 15)
▁2080: 89553: (15, 20)
-: 271: (20, 21)
11: 1554: (21, 23)
-: 271: (23, 24)
30: 1076: (24, 26)
▁Reason: 18037: (29, 36)
▁for: 270: (36, 40)
▁Visit: 3847: (40, 46)
▁Owen: 12980: (47, 52)
▁is: 269: (52, 55)
▁a: 266: (55, 57)
▁63: 7592: (57, 60)
▁y: 2982: (60, 62)
/: 320: (62, 63)
o: 795: (63, 64)
▁male: 2844: (64, 69)
▁here: 422: (69, 74)
▁for: 270: (74, 78)
▁evaluation: 4291: (78, 89)
▁of: 265: (89, 92)
▁treatment: 1035: (92, 102)
.: 260: (102, 103)
▁Do: 771: (103, 106)
in: 547: (106, 108)
▁relatively: 2936: (108, 119)
▁well: 371: (119, 124)
.: 260: (124, 125)
▁Problems: 16067: (129, 138)
▁OA: 27122: (147, 150)
▁L: 1093: (159, 161)
LE: 14360: (161, 163)
-: 271: (163, 164)
PART: 71889: (164, 168)
I: 476: (168, 169)
ALLY: 43993: (169, 173)
▁S: 662: (173, 175)
EVER: 45227: (175, 179)
ED: 4272: (179, 181)
-: 271: (181, 182)
M: 1135: (182, 183)
ULT: 55111: (183, 186)
.: 260: (186, 187)
▁S: 662: (187, 189)
URGE: 9670

In [15]:
label_map = {
    "O": 0,
    "B-DATE": 1,
    "I-DATE": 2,
    "B-PATIENT": 3,
    "I-PATIENT": 4,
    "B-AGE": 5,
    "I-AGE": 6,
    "B-STAFF": 7,
    "I-STAFF": 8,
    "B-PHONE": 9,
    "I-PHONE": 10,
    "B-EMAIL": 11,
    "I-EMAIL": 12,
    "B-ID": 13,
    "I-ID": 14,
    "B-HOSP": 15,
    "I-HOSP": 16,
    "B-PATORG": 17,
    "I-PATORG": 18,
    "B-LOC": 19,
    "I-LOC": 20,
    "B-OTHERPHI": 21,
    "I-OTHERPHI": 22,
}

In [16]:
def pre_process_data(text, spans, input_ids):
    tokens = tokz.convert_ids_to_tokens(input_ids)
    labels = ['O'] * len(input_ids)
    token_positions = tokz(text, return_offsets_mapping=True)['offset_mapping']


    for span in spans:
        start, end, label = span['start'], span['end'], span['label']

        token_start, token_end = None, None

        for idx, (char_start, char_end) in enumerate(token_positions):
            if tokens[idx].startswith('▁'):
                char_start += 1
            # print(tokens[idx], char_start, char_end, start, end)
            if char_start == int(start):
                token_start = idx
            if char_end == int(end):
                token_end = idx
                break
        
        if token_start is not None and token_end is not None:
            # print(token_start, token_end, label)
            labels[token_start] = f'B-{label}'
            for idx in range(token_start + 1, token_end + 1):
                labels[idx] = f'I-{label}'

    # input_ids = tokz.convert_tokens_to_ids(tokens)
    label_ids = [label_map[label] for label in labels]

    return label_ids

In [17]:
tok_ds_processed = tok_ds.add_column('labels', [pre_process_data(text, spans, input_ids) for text, spans, input_ids in zip(tok_ds['text'], tok_ds['spans'], tok_ds['input_ids'])])

In [18]:
test_tok_ds_processed = test_tok_ds.add_column('labels', [pre_process_data(text, spans, input_ids) for text, spans, input_ids in zip(test_tok_ds['text'], test_tok_ds['spans'], test_tok_ds['input_ids'])])

In [19]:
# Print out mapping of the tokens to the labels
tokens = tokz.convert_ids_to_tokens(tok_ds_processed['input_ids'][1])
labels = [list(label_map.keys())[list(label_map.values()).index(label_id)] for label_id in tok_ds_processed['labels'][1]]

for token, label in zip(tokens, labels):
    print(f'{token} - {label}')

[CLS] - O
▁Record - O
▁date - O
: - O
▁20 - B-DATE
61 - I-DATE
- - I-DATE
01 - I-DATE
- - I-DATE
15 - I-DATE
▁ST - B-HOSP
▁LUCI - I-HOSP
A - I-HOSP
▁EMERGENCY - O
▁DEP - O
T - O
▁VISIT - O
▁VU - B-PATIENT
ONG - I-PATIENT
, - I-PATIENT
XO - I-PATIENT
CHI - I-PATIENT
LT - I-PATIENT
▁R - I-PATIENT
▁515 - B-ID
- - I-ID
31 - I-ID
- - I-ID
23 - I-ID
- - I-ID
5 - I-ID
▁VISIT - O
▁DATE - O
: - O
▁01 - B-DATE
/ - I-DATE
15 - I-DATE
/ - I-DATE
61 - I-DATE
▁PRESENT - O
ING - O
▁COMP - O
LAIN - O
T - O
: - O
▁Bleeding - O
. - O
▁HISTORY - O
▁OF - O
▁PRESENT - O
ING - O
▁COMP - O
LAIN - O
T - O
: - O
▁This - O
▁71 - B-AGE
▁year - O
▁old - O
▁female - O
▁status - O
▁post - O
▁cardiac - O
▁cath - O
▁stent - O
▁placement - O
▁and - O
▁then - O
▁later - O
▁EP - O
▁study - O
▁states - O
▁that - O
▁she - O
▁was - O
▁doing - O
▁well - O
▁until - O
▁last - O
▁night - O
▁when - O
▁she - O
▁started - O
▁to - O
▁ooze - O
▁from - O
▁the - O
▁groin - O
▁site - O
. - O
▁She - O
▁did - O
▁not - O
▁apply - O
▁any 

In [20]:
## Create test and validation sets
## test_tok_ds_processed is the test set
tok_dds = tok_ds_processed.train_test_split(test_size=0.25, seed=42)
tok_dds

DatasetDict({
    train: Dataset({
        features: ['text', 'spans', 'meta', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 592
    })
    test: Dataset({
        features: ['text', 'spans', 'meta', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 198
    })
})

### Training the model

In [21]:
from transformers import TrainingArguments, Trainer
import accelerate

In [22]:
bs = 128
epochs = 4

In [23]:
lr = 8e-5

In [25]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=False,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [26]:
from transformers import AutoModelForTokenClassification, AutoConfig


In [27]:
config = AutoConfig.from_pretrained(model_nm, num_labels=len(label_map))


In [28]:
model = AutoModelForTokenClassification.from_pretrained(model_nm, config=config)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a B

In [29]:
trainer = Trainer(model, args, train_dataset=tok_dds['train'], eval_dataset=tok_dds['test'], tokenizer=tokz)

In [30]:
trainer.train()

/Users/manibatra/mambaforge/envs/phi/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/20 [00:00<?, ?it/s]You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [41]:
# tok_dds['test']['labels'][0]
tok_dds['train']['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [39]:
for i in range(len(tok_dds['test']['input_ids'])):
    print(len(tok_dds['test']['input_ids'][i]))

1750
1242
1926
1207
693
1429
1164
300
2554
2549
567
919
391
2314
410
663
821
449
1110
667
697
552
639
646
931
820
665
538
775
990
1012
2312
157
656
902
1790
548
1888
462
646
334
790
1786
1115
1246
698
930
1158
778
782
492
681
1591
723
1335
1593
2339
1400
465
566
602
1505
597
872
2058
1955
1291
865
694
1049
716
2191
541
1416
1066
1040
246
546
726
819
743
528
1193
867
1040
1762
696
961
601
481
431
1537
213
666
378
1346
1479
432
613
589
1953
3064
639
514
347
1816
306
788
361
566
1415
549
538
789
3941
492
735
239
1231
1912
1504
635
748
1023
3569
866
659
394
534
923
1718
1072
590
733
1514
942
742
756
550
1204
403
802
344
895
2251
2171
2353
362
1951
460
556
885
1293
584
1539
1385
1406
353
1543
704
1392
1495
1101
2619
1114
1002
950
932
1234
743
1384
1118
1125
675
906
1091
1124
400
567
628
659
1175
1577
603
617
831
1914
713
1033
418
630
740
607
477
658
1474
420
1545
